<a href="https://colab.research.google.com/github/ivnka24/PySpark_Project/blob/main/PySpark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.7.13


### Install Python dan PySpark SQL

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

### Load File Dataset

In [8]:
!wget --continue https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-06-06 02:29:56--  https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-06-06 02:29:57 (17.6 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [9]:
df = spark.read.json("/tmp/sample_books.json")

In [10]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



### Praktikum

In [11]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



In [12]:
df.count()

13

In [13]:
df.select("title", "price", "year_written").show(5)


+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [14]:
df_filtered1 = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered1.select("title", "price", "year_written").show(50, False)


+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [15]:
from pyspark.sql.functions import max
df_filtered = df.select("year_written","title","price")

# Find the costliest book

maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  29.0
+-------------------+-----+
|title              |price|
+-------------------+-----+
|A Room of One's Own|29.0 |
+-------------------+-----+



### Tugas Praktikum

In [16]:
#1.tampilkan data buku paling murah
from pyspark.sql.functions import min
# Find the costliest book
minValue = df_filtered.agg(min("price")).collect()[0][0]
print("minValue: ",minValue)

df_filtered.select("title","price").filter(df.price == minValue).show(20, False)

minValue:  5.75
+-----------+-----+
|title      |price|
+-----------+-----+
|Bleak House|5.75 |
+-----------+-----+



In [17]:
#2.Tampilkan jumlah terbit buku dikategorikan setiap tahun ditulis!
from pyspark.sql import functions as s 
df.groupby("year_written").agg(s.count("year_written").alias("jumlah")).show()

+------------+------+
|year_written|jumlah|
+------------+------+
|        1967|     1|
|        1937|     1|
|        1862|     1|
|        1999|     1|
|        1875|     1|
|        1925|     1|
|        1814|     1|
|        1865|     2|
|        1922|     1|
|        2000|     1|
|        1870|     1|
|        1603|     1|
+------------+------+



In [18]:
 #3.Tampilkan data buku termahal tiap tahun penulisannya!
from pyspark.sql.functions import max # Find the costliest book
maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)

df_filtered.select("year_written","price").filter(df.price == maxValue).show(20, False)


maxValue:  29.0
+------------+-----+
|year_written|price|
+------------+-----+
|1922        |29.0 |
+------------+-----+



In [19]:
 #4.Tampilkan data buku termurah tiap tahun penulisannya!
from pyspark.sql.functions import min # Find the costliest book
minValue = df_filtered.agg(min("price")).collect()[0][0]
print("minValue: ",minValue)

df_filtered.select("year_written","price").filter(df.price == minValue).show(20, False)

minValue:  5.75
+------------+-----+
|year_written|price|
+------------+-----+
|1870        |5.75 |
+------------+-----+

